# I used Arduino IDE for this Project.

This code enables fingerprint enrollment using an Adafruit fingerprint sensor with an ESP32 or Arduino. It captures a student's fingerprint, processes it into a template, and stores it in the sensor's memory under a unique ID (roll number). The system ensures fingerprint matching before saving, allowing for secure and reliable identification.

# Steps:

## 1. Include Required Library
We include the Adafruit_Fingerprint.h library to interface with the fingerprint sensor.

In [ ]:
#include <Adafruit_Fingerprint.h>

## 2. Define Serial Communication
The fingerprint sensor communicates with the Arduino using serial communication.

- If using an Arduino Uno or ESP8266, we use SoftwareSerial on pins 2 (RX) and 3 (TX).
- If using a board with hardware serial support (like Arduino Mega), we use Serial2.

In [ ]:
#if (defined(__AVR__) || defined(ESP8266)) && !defined(__AVR_ATmega2560__)
SoftwareSerial mySerial(2, 3); // RX = 2, TX = 3
#else
#define mySerial Serial2
#endif


## 3. Initialize Fingerprint Sensor
We create an instance of Adafruit_Fingerprint and define an id variable to store the fingerprint ID.

In [ ]:
Adafruit_Fingerprint finger = Adafruit_Fingerprint(&mySerial);
uint8_t id;

## 4. Setup Function (Runs Once)
- Initializes Serial communication for debugging.
- Initializes the fingerprint sensor.
- Displays sensor parameters.



In [ ]:
void setup()
{
  Serial.begin(115200);
  while (!Serial);  // Waits for serial connection (useful for certain boards)
  delay(100);
  Serial.println("\n\nAdafruit Fingerprint sensor enrollment");

  // Start fingerprint sensor communication
  finger.begin(57600);

  if (finger.verifyPassword()) {
    Serial.println("Found fingerprint sensor!");
  } else {
    Serial.println("Did not find fingerprint sensor :(");
    while (1) { delay(1); } // Halt execution
  }

  Serial.println(F("Reading sensor parameters"));
  finger.getParameters();
  Serial.print(F("Status: 0x")); Serial.println(finger.status_reg, HEX);
  Serial.print(F("Sys ID: 0x")); Serial.println(finger.system_id, HEX);
  Serial.print(F("Capacity: ")); Serial.println(finger.capacity);
  Serial.print(F("Security level: ")); Serial.println(finger.security_level);
  Serial.print(F("Device address: ")); Serial.println(finger.device_addr, HEX);
  Serial.print(F("Packet len: ")); Serial.println(finger.packet_len);
  Serial.print(F("Baud rate: ")); Serial.println(finger.baud_rate);
}


## 5. Read a Number from Serial
This function waits for user input and reads an integer from the serial monitor.

In [ ]:
uint8_t readnumber(void) {
  uint8_t num = 0;

  while (num == 0) {
    while (!Serial.available()); // Wait for user input
    num = Serial.parseInt();
  }
  return num;
}


## 6. Main Loop (Runs Continuously)
- Asks the user to enter a student's roll number (ID) for fingerprint enrollment.
- Calls getFingerprintEnroll() to enroll the fingerprint.

In [ ]:
void loop() {
  Serial.println("Ready to enroll a fingerprint!");
  Serial.println("Please Enter the roll number of the Student (capacity: 127) you want to save his finger...");
  
  id = readnumber();
  if (id < 1 || id > 127) {
    return; // Invalid ID, do nothing
  }

  Serial.print("Enrolling ID #");
  Serial.println(id);

  while (!getFingerprintEnroll()); // Keep trying until enrollment is successful
}


## 7. Enroll Fingerprint Function
This function guides the user through the fingerprint enrollment process.

### 7.1. Wait for Fingerprint
- Waits until a finger is placed.
- Displays error messages if any issue occurs.

In [ ]:
uint8_t getFingerprintEnroll() {
  int p = -1;
  Serial.print("Waiting for valid finger to enroll as #"); Serial.println(id);
  
  while (p != FINGERPRINT_OK) {
    p = finger.getImage();
    switch (p) {
      case FINGERPRINT_OK:
        Serial.println("Image taken");
        break;
      case FINGERPRINT_NOFINGER:
        Serial.print(".");
        break;
      case FINGERPRINT_PACKETRECIEVEERR:
        Serial.println("Communication error");
        break;
      case FINGERPRINT_IMAGEFAIL:
        Serial.println("Imaging error");
        break;
      default:
        Serial.println("Unknown error");
        break;
    }
  }


### 7.2. Convert Image to Template
- Converts the fingerprint image into a template.


In [ ]:
  p = finger.image2Tz(1);
  switch (p) {
    case FINGERPRINT_OK:
      Serial.println("Image converted");
      break;
    case FINGERPRINT_IMAGEMESS:
      Serial.println("Image too messy");
      return p;
    case FINGERPRINT_PACKETRECIEVEERR:
      Serial.println("Communication error");
      return p;
    case FINGERPRINT_FEATUREFAIL:
    case FINGERPRINT_INVALIDIMAGE:
      Serial.println("Could not find fingerprint features");
      return p;
    default:
      Serial.println("Unknown error");
      return p;
  }


### 7.3. Ask for Second Scan
The user removes and places the same finger again.

In [ ]:
  Serial.println("Remove finger");
  delay(2000);
  
  p = 0;
  while (p != FINGERPRINT_NOFINGER) {
    p = finger.getImage();
  }

  Serial.print("ID "); Serial.println(id);
  p = -1;
  Serial.println("Place same finger again");

  while (p != FINGERPRINT_OK) {
    p = finger.getImage();
    switch (p) {
      case FINGERPRINT_OK:
        Serial.println("Image taken");
        break;
      case FINGERPRINT_NOFINGER:
        Serial.print(".");
        break;
      case FINGERPRINT_PACKETRECIEVEERR:
        Serial.println("Communication error");
        break;
      case FINGERPRINT_IMAGEFAIL:
        Serial.println("Imaging error");
        break;
      default:
        Serial.println("Unknown error");
        break;
    }
  }


### 7.4. Convert Second Image
- Converts the second image into a template.

In [ ]:
  p = finger.image2Tz(2);
  switch (p) {
    case FINGERPRINT_OK:
      Serial.println("Image converted");
      break;
    case FINGERPRINT_IMAGEMESS:
      Serial.println("Image too messy");
      return p;
    case FINGERPRINT_PACKETRECIEVEERR:
      Serial.println("Communication error");
      return p;
    case FINGERPRINT_FEATUREFAIL:
    case FINGERPRINT_INVALIDIMAGE:
      Serial.println("Could not find fingerprint features");
      return p;
    default:
      Serial.println("Unknown error");
      return p;
  }


### 7.5. Create and Store Fingerprint Model
- The two templates are matched.
- If they match, the fingerprint is stored in memory.

In [ ]:
  Serial.print("Creating model for #"); Serial.println(id);
  p = finger.createModel();

  if (p == FINGERPRINT_OK) {
    Serial.println("Prints matched!");
  } else if (p == FINGERPRINT_ENROLLMISMATCH) {
    Serial.println("Fingerprints did not match");
    return p;
  } else {
    Serial.println("Unknown error");
    return p;
  }

  Serial.print("ID "); Serial.println(id);
  p = finger.storeModel(id);

  if (p == FINGERPRINT_OK) {
    Serial.println("Stored!");
  } else if (p == FINGERPRINT_BADLOCATION) {
    Serial.println("Could not store in that location");
    return p;
  } else if (p == FINGERPRINT_FLASHERR) {
    Serial.println("Error writing to flash");
    return p;
  } else {
    Serial.println("Unknown error");
    return p;
  }

  return true;
}


## Summary
This Arduino sketch:
- ✔ Initializes and configures the fingerprint sensor.
- ✔ Prompts the user for an ID.
- ✔ Guides the user through fingerprint scanning (two scans).
- ✔ Matches the fingerprint and stores it in memory.